In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = 500

In [57]:
# df.to_csv('FE_fights', index=False)

# Feature Egineering 
### - Create time from last fight
### - ordinal encode Max_round
### - HomeTown will be logitude latitude 
### - Vectorize Name might help 

In [3]:
df = pd.read_csv('clean_fights')
df.head()

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
0,0,0,38.0,193.0,Hounslow England,808,Amsterdam The Netherlands,Alistair Overeem,120.0,02/03/2014,646,4580,3,3,0,39.0,190.0,"Las Vegas, Nevada USA",377,"Las Vegas, Nevada USA",Frank Mir,119.0,DEC,blue,262.9,Cruiserweight
1,0,0,36.0,172.0,"Chicago, Illinois United States",1054,"Chicago, Illinois United States",Ricardo Lamas,65.0,02/03/2014,646,4589,5,5,0,32.0,170.0,Manaus Brazil,1052,Rio de Janeiro Brazil,Jose Aldo,65.0,DEC,red,143.0,Bantamweight
2,0,0,39.0,167.0,"Isla Vista , California USA",959,"Sacramento, California USA",Urijah Faber,61.0,02/03/2014,646,4590,1,5,0,31.0,167.0,Natal Brazil,1527,Rio de Janeiro Brazil,Renan Barao,61.0,KO/TKO,red,134.2,Flyweight
3,0,0,33.0,167.0,"San Diego, CA USA",1056,"San Diego, CA USA",Danny Martinez,56.0,02/03/2014,646,4605,3,3,0,37.0,160.0,"San Jose, California USA",1253,"Tucson, Arizona USA",Chris Cariaso,56.0,DEC,red,123.2,Strawwweight
4,0,0,36.0,185.0,Southampton England,2005,Southampton England,Tom Watson,84.0,02/03/2014,646,4631,3,3,0,37.0,182.0,"Englewood, NJ USA",464,"Brick, NJ USA",Nick Catone,84.0,DEC,red,184.8,Super Welterweight


## Create time since last fight 
- group each fighter (need to group so that both corners are looked at for time line)
- order based on date
- use an array the size of the data set
- replace none values to the index of that fighter 
- do this for each corner

In [4]:
date = pd.to_datetime(df.Date)
df.Date = date
date.iloc[0]

Timestamp('2014-02-03 00:00:00')

In [ ]:
R_group = df.groupby('R_ID')
# len(R_group.groups)
R_group.groups

In [ ]:
B_group = df.groupby('B_ID')
B_group.groups

In [7]:
129 in R_group.groups

True

In [27]:
R_i = R_group.indices[129].tolist()

In [28]:
B_i = B_group.indices[129].tolist()
R_i + B_i

[526, 1102, 128, 161, 638, 851, 1306]

In [56]:
df.iloc[R_group.indices[129]]

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class,R_diff,B_diff
526,3,0,33.0,187.0,Tehran Iran,786,"Leiden, Holland The Netherlands",Gegard Mousasi,84.0,2015-01-27,702,5132,1,3,2,48.0,180.0,"Downey, California USA",129,"Temecula, California USA",Dan Henderson,84.0,KO/TKO,blue,184.8,Super Welterweight,248 days,141 days
1102,3,0,40.0,175.0,Matanzas Cuba,768,"Matanzas, Cuba United States",Hector Lombard,84.0,2017-02-13,774,6108,2,3,5,48.0,180.0,"Downey, California USA",129,"Temecula, California USA",Dan Henderson,84.0,KO/TKO,red,184.8,Super Welterweight,125 days,0 days


In [10]:
R_delta = np.empty(df.shape[0], dtype=pd.Timestamp)
B_delta = np.empty(df.shape[0], dtype=pd.Timestamp)

- iterate through all fighter IDs
- Use ID to get the Red corner and Blue corners into variables
- Create a data frame for each fighter containing the date of the fight and whether which corner they were in and the index and 
- Iterate through the dataframe and create a time delta from last match column
- Add that to corresponding index and corresponding array

In [11]:
# Create list of all IDs
IDs = pd.concat([df.R_ID, df.B_ID])
IDs = IDs.drop_duplicates()

In [12]:
# Create column names for empty DataFrame
columns = ['corner', 'Date', 'diff', 'index'] #, 'index']

In [49]:
for ID in IDs:
    temp_df = pd.DataFrame(columns=columns)
    
    if ID in R_group.groups:
        r_corner = ['r'] * len(R_group.groups[ID])
        r_date = df.iloc[R_group.groups[ID]].Date.tolist()
        r_index = R_group.indices[ID].tolist()
        
    if ID in B_group.groups:
        b_corner = ['b'] * len(B_group.groups[ID])
        b_date = df.iloc[B_group.groups[ID]].Date.tolist()
        b_index = B_group.indices[ID].tolist()
    
    temp_df['corner'] = r_corner + b_corner
    temp_df['Date'] = r_date + b_date
    temp_df['index'] = r_index + b_index
    temp_df = temp_df.sort_values(by=['Date'])
    temp_df['diff'] = temp_df['Date'].diff().fillna(pd.to_timedelta(0))
    
#     print(temp_df.head())

    for _, row in temp_df.iterrows():
        if row.corner == 'r':
            R_delta[row['index']] = row['diff']

        elif row['corner'] == 'b':
            B_delta[row['index']] = row['diff']
    

In [55]:
df['B_diff'] = B_delta

In [30]:
pd.Series(sorted(df.iloc[[1, 20, 50]].Date)).diff()

0       NaT
1   18 days
2   13 days
dtype: timedelta64[ns]

In [70]:
c = ['r'] * 10
c2 = ['b'] * 3
c3 = c + c2
# dfdf = pd.DataFrame({'corner':c3})
# dfdf
c3

['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'b', 'b', 'b']

## Vectorize names with word to vec
 - I don't think this will work because there's already an ID

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

In [ ]:
names = df.head().B_Name.values

In [ ]:
names

In [ ]:
tokens = [word_tokenize(x) for x in names]
tokens

In [ ]:
[Word2Vec(x) for x in tokens]

## Use Geopy to add longitude and latitude based off HomeTown

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='pwalis', timeout=3)
location = geolocator.geocode("Brazil", addressdetails=True)


In [ ]:
one = location


In [ ]:
import time
start = time.process_time()
[geolocator.geocode(x) for x in df.R_HomeTown.head(30)]
print(time.process_time() - start)

In [ ]:
R_geo = []
i = 0
for x in df.R_HomeTown:
    i+=1
    print(i)
    geo_info = geolocator.geocode(x)
    R_geo.append(geo_info)
# honestly I just want a green square

A lot of places aren't getting recognized as a location I should look into how the api expects the input
Research how to handle different errors 
if statements based on the error, If it can't get the location info add an entry into the dict that has the hometown string

In [ ]:
from geopy.exc import GeocoderTimedOut
i = 0
R_geo = dict()
for x in df.R_HomeTown:
    i+=1 
    try:
        location = geolocator.geocode(x).raw
        R_geo.update({i:location})
    except Exception as e:
        print("Error: geocode failed on input %s with message %s"%(x, e))
        

In [ ]:
len(R_geo)

In [ ]:
i=0
while i<1162:
    i+=1
    if len(R_geo[i])<10:
        print(R_geo[i])
        

In [ ]:
R_geo.keys()